<a href="https://colab.research.google.com/github/alvumu/TFM/blob/main/Llama2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://huggingface.co/NousResearch/Llama-2-7b-hf

Cloning into 'Llama-2-7b-hf'...
remote: Enumerating objects: 33, done.
remote: Total 33 (delta 0), reused 0 (delta 0), pack-reused 33
Unpacking objects: 100% (33/33), 491.74 KiB | 5.23 MiB/s, done.


In [ ]:
from transformers import AutoTokenizer,AutoModelForCauslLM, pipeline
import torch
from peft import PeftModel
import csv


In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "NousResearch/Llama-2-7b-hf",
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
)



In [ ]:
model = PeftModel.from_pretrained(model, "LLMTrainedModel/1epochModel")  # Directory that contains the fine tuned model
model = model.merge_and_unload()

# Assuming `model` and `tokenizer` are your loaded model and tokenizer
generator = pipeline("text-generation", model=model, tokenizer="LLMTrainedModel/1epochModel/")


In [ ]:
# Example prompt
prompt = f'''###HUMAN: Your task is to process the following fragment of a medical note, retrieving and listing all SNOMED annotations found inside. You will return an "EMPTY: No SNOMED annotations found in this fragment" if there are none. For any that you find, you will return a list of: "text {annotation}", where 'text' is the part of the text that corresponds to the annotation, and 'annotation' is the corresponding SNOMED-CT annotation. This is the text to analyze:
###TEXT: History of Present Illness:
Mr. ___ is a ___ man who had severe biliary
pancreatitis resulting in pancreatic necrosis for which he was
treated with nasojejunal feedings and pancreatic rest.  He had
initially had multisystem organ failure, which improved. Mr.
___ has a large postnecrotic pseudocyst, which has been
drained through a minimally invasive approach into his GI tract.
 He has some debris, but this is not currently infected. The
patient was followed by Dr. ___ in his ___
clinic to discuss cholecystectomy. After discussion of all
risks, benefits and possible outcomes, patient was scheduled for
elective cholecystectomy on ___.'''
